In [7]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Bag of Words로 Classification 해보기

In [9]:
# 문장이 spanish인지 english인지 classification 하는 문제입니다.

data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

# word들을 integer 값으로 mapping 해줍니다.
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2


class BoWClassifier(nn.Module):  # inheriting from nn.Module!

    def __init__(self, num_labels, vocab_size):
        # 잊지말기 !!! nn.Module 상속 후 super로 불러오기
        super(BoWClassifier, self).__init__()

        # 간단한 MLP 모델입니다. (layer 하나)
        # BoW이기 때문에 voca_size가 곧 벡터의 길이가 됩니다.
        self.linear = nn.Linear(vocab_size, num_labels)

    def forward(self, bow_vec):
        # log softmax를 거친 vector의 확률값을 뱉어냅니다.
        return F.log_softmax(self.linear(bow_vec))


def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)


def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])


model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

for param in model.parameters():
    print(param) # W, b의 값을 차례대로 출력합니다.

sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)
print(bow_vector[0])
log_probs = model(autograd.Variable(bow_vector))
print(log_probs)

{'en': 3, 'No': 9, 'buena': 14, 'it': 7, 'at': 22, 'sea': 12, 'cafeteria': 5, 'Yo': 23, 'la': 4, 'to': 8, 'creo': 10, 'is': 16, 'a': 18, 'good': 19, 'get': 20, 'idea': 15, 'que': 11, 'not': 17, 'me': 0, 'on': 25, 'gusta': 1, 'lost': 21, 'Give': 6, 'una': 13, 'si': 24, 'comer': 2}
Parameter containing:

Columns 0 to 9 
-0.1342 -0.1708  0.0709  0.1419 -0.1424  0.0092 -0.0419 -0.0775 -0.0104  0.0061
-0.1135 -0.1588 -0.0201 -0.1071 -0.0779 -0.0400 -0.1216 -0.0747 -0.1461 -0.0343

Columns 10 to 19 
 0.0550  0.1906  0.1876  0.0392  0.1406 -0.0721 -0.0235  0.1739  0.0083  0.0809
 0.0849 -0.1783 -0.1777 -0.0166 -0.0925  0.0495  0.0466  0.0962 -0.0931  0.1070

Columns 20 to 25 
 0.1462  0.0792  0.0019 -0.0234  0.1682 -0.1821
 0.1481  0.0124 -0.1238 -0.1128 -0.1079 -0.0308
[torch.FloatTensor of size 2x26]

Parameter containing:
-0.1329
 0.0340
[torch.FloatTensor of size 2]


 1
 1
 1
 1
 1
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 26]

Variable con

In [10]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

In [13]:
# training 전에 test를 해보겠습니다.
for instance, label in test_data:
    print make_bow_vector(instance, word_to_ix)
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

# training 전의 creo에 대한 log probability를 찍어봅니다
print(next(model.parameters())[:, word_to_ix["creo"]])

# Negative loglikelihood loss를 사용합니다.
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 보통 100 epoch는 실제 데이터에 적합하지 않습니다. 약 5~30의 epoch수를 사용합니다.
for epoch in range(100):
    for instance, label in data:
        # Pytorch는 gradient가 중첩되기때문에 매 step마다 gradient를 초기화 해주어야 합니다.
        model.zero_grad()

        # Variable로 input vector와 label들을 wrap 해줍니다.
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
        target = autograd.Variable(make_target(label, label_to_ix))

        # forward pass로 계산해줍니다.
        log_probs = model(bow_vec)

        # loss를 계산하고 gradient를 계산한 후에 optimizer로 gradient를 업데이트 하는 과정입니다.
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

# 다시 model에 test data를 넣어봅니다.
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

# creo는 원래 spanish인데, weight값이 spanish쪽이 높아진 것으로 보아 잘 학습된 것을 알 수 있습니다.
print(next(model.parameters())[:, word_to_ix["creo"]])
print next(model.parameters())



Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     1     1     0

Columns 13 to 25 
    0     0     0     0     0     0     0     0     0     0     1     1     0
[torch.FloatTensor of size 1x26]

Variable containing:
-0.0827 -2.5335
[torch.FloatTensor of size 1x2]



Columns 0 to 12 
    1     0     0     0     0     0     0     1     0     0     0     0     0

Columns 13 to 25 
    0     0     0     1     0     0     0     0     1     0     0     0     1
[torch.FloatTensor of size 1x26]

Variable containing:
-3.4761 -0.0314
[torch.FloatTensor of size 1x2]

Variable containing:
 0.5057
-0.3658
[torch.FloatTensor of size 2]

Variable containing:
-0.0712 -2.6770
[torch.FloatTensor of size 1x2]

Variable containing:
-3.6942 -0.0252
[torch.FloatTensor of size 1x2]

Variable containing:
 0.5414
-0.4015
[torch.FloatTensor of size 2]

Parameter containing:

Columns 0 to 9 
-0.2517  0.3950  0.6368  0.7077  0.4234  0.5750 -0.7252 -1.0683 -1.0012  0.1849
 0.0039 -

### Word Embedding 후 n-gram LM 만들어보기

In [14]:
word_to_ix = {"hello": 0, "world": 1}
# tf.get_varaible("embedding_table", [voca_size, embedding_size]) 와 동일합니다.
# 즉, embedding table을 만듭니다. parameters는 차례로 vocab(characters)의 unique 갯수, embedding vector dimension 입니다.
embeds = nn.Embedding(2, 5)

# lookup은 indexing 하면 됩니다.
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed)

Variable containing:
-0.8690  1.6708 -0.1012  0.6869 -1.1280
[torch.FloatTensor of size 1x5]



In [15]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

# 셰익스피어 모네 data 입니다.
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# ((2번째 전의 단어, 1번째 전의 단어), 타겟 단어)로 튜플을 만듭니다.
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]


# 3개만 프린트 해보겠습니다.
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [16]:
# set이라는 좋은 함수가 있다는 걸 오늘 처음 알았네요. 자동으로 unique한 애들만 골라줍니다
vocab = set(test_sentence)
print vocab
word_to_ix = {word: i for i, word in enumerate(vocab)}

set(['all', 'brow,', 'being', 'couldst', 'treasure', 'Proving', 'to', 'field,', 'worth', 'his', 'thine!', 'lies,', 'Where', 'dig', 'succession', 'small', 'praise.', 'where', 'old', 'fair', 'see', 'Thy', "deserv'd", 'sum', 'shall', 'forty', 'new', 'be', 'asked,', 'days;', 'This', 'Were', 'say,', 'by', 'on', 'thou', 'of', 'thine', 'own', 'gazed', 'within', 'When', 'art', 'now,', 'trenches', "feel'st", 'much', 'more', 'held:', 'count,', 'it', 'warm', "beauty's", 'child', 'an', "youth's", 'And', 'made', 'How', "'This", 'praise', 'were', 'eyes,', 'my', 'old,', 'and', 'use,', 'mine', 'deep', 'livery', 'To', 'shame,', 'in', 'Then', 'all-eating', 'sunken', 'Shall', 'make', 'when', 'thriftless', 'answer', 'lusty', 'beauty', 'besiege', 'weed', 'Will', "excuse,'", 'blood', 'winters', 'a', 'thy', 'proud', 'cold.', "totter'd", 'so', 'the', 'If'])


In [36]:
class TrigramClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_size, context_size):
        super(TrigramClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.fc = nn.Linear(context_size * embedding_size, 128)
        self.fc2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        
        embed = self.embedding(inputs).view((1,-1))
        out = F.relu(self.fc(embed))
        out = self.fc2(out)
        
        return out
        
model = TrigramClassifier(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)

In [39]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    loss_per_epoch = 0.0
    for _input, label in trigrams:
        model.zero_grad()
        _input = autograd.Variable(torch.LongTensor([word_to_ix[w] for w in _input]))
        
        prob = model(_input)
        label = autograd.Variable(torch.LongTensor([word_to_ix[label]]))
        
        loss = loss_function(prob, label)
        
        loss.backward()
        optimizer.step()
        
        
        loss_per_epoch += loss.data[0]
        
    print loss_per_epoch
    


2.80429686469
2.79989315488
2.81764824557
2.80762083901
2.80451477984
2.80253405706
2.80112398949
2.80018622932
2.79969908038
2.79953677681


### LSTM model 만들어보기

In [40]:
lstm = nn.LSTM(3,3)

In [42]:
inputs = [autograd.Variable(torch.randn((1,3))) for _ in range(5)]

In [43]:
inputs

[Variable containing:
  0.4438  0.3946  1.4681
 [torch.FloatTensor of size 1x3], Variable containing:
 -0.1383 -0.7589 -0.7654
 [torch.FloatTensor of size 1x3], Variable containing:
 -0.6630 -0.6874 -0.5107
 [torch.FloatTensor of size 1x3], Variable containing:
  1.8368 -0.2794 -0.8937
 [torch.FloatTensor of size 1x3], Variable containing:
 -0.6421  0.4721 -2.9939
 [torch.FloatTensor of size 1x3]]

In [45]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# weight가 어떻게 변하는지 보기위해 dimension을 작게 가져감
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'Everybody': 5, 'ate': 2, 'apple': 4, 'that': 7, 'read': 6, 'dog': 1, 'book': 8, 'the': 3, 'The': 0}


In [55]:
class LSTMTagger(nn.Module):
    def __init__(self, vocab_size, embedding_size, num_units, target_size):
        # 무조건 합니다
        super(LSTMTagger, self).__init__()
        # lstm num_units (hidden dimesion) 설정
        self.num_units = num_units
        # embedding table을 만들어줍니다.
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        # lstm을 선언합니다. [input dimension, hidden dimension]
        self.lstm = nn.LSTM(embedding_size, num_units)
        # classification용 output linear layer입니다.
        self.output = nn.Linear(num_units, target_size)
        
        # pytorch는 lstm cell의 hidden을 따로 선언해주고 처음에 initialize 해야 합니다.
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # (h_n, c_n) pair 입니다.
        return (autograd.Variable(torch.zeros(1,1,self.num_units)), autograd.Variable(torch.zeros(1,1,self.num_units)))
        
    
    def forward(self, inputs):
        # embedding table에서 input vector를 꺼내옵니다. [seq_len, batch_size, embedding_size]
        embed = self.embedding(inputs)
        
        # output: [seq_len, batch_size, num_units], self.hidden: [num_layers, batch_size, num_units]
        output, self.hidden = self.lstm(embed.view(len(inputs), 1, -1), self.hidden)
        
        # softmax layer
        output = self.output(output.view(len(inputs),-1))
        output = F.log_softmax(output)
        
        return output
        

In [50]:
model = LSTMTagger(len(word_to_ix), EMBEDDING_DIM, HIDDEN_DIM, len(tag_to_ix))

In [56]:
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(300):
    for inputs, targets in training_data:
        # index의 list로 변환
        inputs = prepare_sequence(inputs, word_to_ix)
        targets = prepare_sequence(targets, tag_to_ix)
        
        # 무조건 해줍니다.
        model.zero_grad()
        
        # lstm hidden state를 초기화해줍니다. (이거 질문)
        model.hidden = model.init_hidden()

        output = model(inputs)

        loss = loss_function(output, targets)
        loss.backward()
        optimizer.step()

Variable containing:
 1.1204 -0.0969  0.5942 -0.8957  1.2804 -0.0385
-0.3686  2.4810  0.4184  0.6698  2.1916  0.6093
-0.6337  1.4208 -0.2956  0.2502  1.6885  2.0661
 0.1398 -0.3444  1.1087  2.7113 -1.5817 -0.1492
-2.4102 -0.9229  0.6483 -1.3444 -0.3385  1.7318
[torch.FloatTensor of size 5x6]

Variable containing:
-0.1712 -2.1370 -3.2351
-2.9756 -0.0917 -3.3064
-3.8382 -2.6974 -0.0931
-0.3800 -1.4210 -2.5949
-3.2260 -0.0787 -3.3251
[torch.FloatTensor of size 5x3]

Variable containing:
-1.3392  1.1489  0.5815  0.7406  0.1378  0.2284
 0.7643  0.8523 -1.2147  0.5808 -0.2415  0.5530
 0.3659 -0.3351 -0.3096 -0.8295  0.5374 -1.0464
-0.4735  1.0864  0.6009  0.1118 -1.1463  0.4832
[torch.FloatTensor of size 4x6]

Variable containing:
-2.7524 -0.1313 -2.8251
-2.9768 -3.0037 -0.1060
-0.1421 -2.4492 -3.0760
-2.8376 -0.0811 -3.9455
[torch.FloatTensor of size 4x3]

Variable containing:
 1.1189 -0.0953  0.5926 -0.8973  1.2821 -0.0369
-0.3702  2.4826  0.4201  0.6715  2.1933  0.6076
-0.6320  1.4191 -0.

Variable containing:
-0.1180 -2.4672 -3.6328
-3.2544 -0.0669 -3.6472
-4.0956 -3.1519 -0.0613
-0.3110 -1.5665 -2.8387
-3.3778 -0.0625 -3.6319
[torch.FloatTensor of size 5x3]

Variable containing:
-1.3757  1.1851  0.6169  0.7000  0.1549  0.2651
 0.7289  0.8886 -1.2499  0.5748 -0.2051  0.5888
 0.3298 -0.2979 -0.3476 -0.8661  0.5744 -1.0093
-0.5094  1.1230  0.6286  0.1455 -1.1081  0.5194
[torch.FloatTensor of size 4x6]

Variable containing:
-3.0313 -0.0944 -3.1738
-3.3989 -3.2747 -0.0739
-0.1026 -2.6979 -3.5005
-3.1057 -0.0587 -4.4072
[torch.FloatTensor of size 4x3]

Variable containing:
 1.1354 -0.0576  0.5603 -0.9339  1.3188  0.0018
-0.4055  2.5182  0.4557  0.7082  2.2350  0.5632
-0.5955  1.3830 -0.3322  0.2916  1.7260  2.1027
 0.1772 -0.3831  1.1473  2.7363 -1.5594 -0.1865
-2.4479 -0.8829  0.6871 -1.3828 -0.3735  1.7688
[torch.FloatTensor of size 5x6]

Variable containing:
-0.1164 -2.4793 -3.6470
-3.2638 -0.0661 -3.6586
-4.1048 -3.1667 -0.0604
-0.3086 -1.5726 -2.8470
-3.3816 -0.0620 -3.

Variable containing:
-0.0837 -2.7822 -3.9963
-3.4992 -0.0512 -3.9278
-4.3279 -3.5268 -0.0435
-0.2430 -1.7696 -3.0936
-3.3985 -0.0534 -3.9869
[torch.FloatTensor of size 5x3]

Variable containing:
-1.4071  1.2157  0.6441  0.6592  0.2088  0.2984
 0.7010  0.9191 -1.2770  0.6138 -0.1741  0.6179
 0.2978 -0.2641 -0.3821 -0.8980  0.6068 -0.9738
-0.5399  1.1550  0.6303  0.1737 -1.0708  0.5504
[torch.FloatTensor of size 4x6]

Variable containing:
-3.2948 -0.0699 -3.4920
-3.7418 -3.4946 -0.0556
-0.0747 -2.9702 -3.8795
-3.3042 -0.0461 -4.7857
[torch.FloatTensor of size 4x3]

Variable containing:
 1.1236 -0.0239  0.5469 -0.9652  1.3509  0.0385
-0.4343  2.5469  0.4852  0.7413  2.2786  0.5159
-0.5613  1.3468 -0.3604  0.3332  1.7588  2.1290
 0.2039 -0.4153  1.1783  2.6853 -1.4890 -0.2130
-2.4778 -0.8389  0.7243 -1.3917 -0.3933  1.7992
[torch.FloatTensor of size 5x6]

Variable containing:
-0.0827 -2.7932 -4.0089
-3.5081 -0.0507 -3.9372
-4.3361 -3.5396 -0.0430
-0.2402 -1.7795 -3.1054
-3.3952 -0.0532 -4.

Variable containing:
-0.0612 -3.0732 -4.3378
-3.7510 -0.0396 -4.1765
-4.5573 -3.8471 -0.0323
-0.1626 -2.1215 -3.4989
-3.2698 -0.0504 -4.5006
[torch.FloatTensor of size 5x3]

Variable containing:
-1.4347  1.2425  0.6648  0.6261  0.2564  0.3304
 0.6776  0.9454 -1.2983  0.6526 -0.1472  0.6424
 0.2662 -0.2331 -0.4086 -0.9245  0.6342 -0.9381
-0.5677  1.1842  0.6413  0.2151 -1.0347  0.5779
[torch.FloatTensor of size 4x6]

Variable containing:
-3.5658 -0.0527 -3.7699
-4.0393 -3.7038 -0.0432
-0.0537 -3.2691 -4.2497
-3.4986 -0.0369 -5.1204
[torch.FloatTensor of size 4x3]

Variable containing:
 1.1096  0.0050  0.5559 -0.9919  1.3796  0.0723
-0.4601  2.5715  0.5116  0.7716  2.3199  0.4743
-0.5290  1.3117 -0.3845  0.3730  1.7910  2.1515
 0.2129 -0.4348  1.1929  2.6163 -1.4130 -0.2184
-2.5051 -0.7817  0.7632 -1.3297 -0.3673  1.8449
[torch.FloatTensor of size 5x6]

Variable containing:
-0.0605 -3.0829 -4.3497
-3.7602 -0.0393 -4.1849
-4.5655 -3.8570 -0.0320
-0.1600 -2.1359 -3.5150
-3.2677 -0.0502 -4.

Variable containing:
-0.0464 -3.3290 -4.6510
-4.0112 -0.0309 -4.3979
-4.7843 -4.1018 -0.0252
-0.1041 -2.5327 -3.9428
-3.3268 -0.0437 -4.9897
[torch.FloatTensor of size 5x3]

Variable containing:
-1.4577  1.2647  0.6815  0.6024  0.2760  0.3583
 0.6575  0.9678 -1.3143  0.6879 -0.1250  0.6628
 0.2355 -0.2056 -0.4198 -0.9422  0.6560 -0.9047
-0.5930  1.2090  0.6889  0.2634 -1.0027  0.6015
[torch.FloatTensor of size 4x6]

Variable containing:
-3.8375 -0.0405 -4.0081
-4.2985 -3.9029 -0.0344
-0.0395 -3.5554 -4.5848
-3.7248 -0.0290 -5.4171
[torch.FloatTensor of size 4x3]

Variable containing:
 1.0736  0.0282  0.5822 -1.0136  1.4043  0.1029
-0.4827  2.5919  0.5338  0.7972  2.3537  0.4511
-0.5058  1.2892 -0.4023  0.3987  1.8147  2.1733
 0.1992 -0.4352  1.1891  2.5655 -1.3556 -0.1991
-2.5322 -0.7273  0.7927 -1.2582 -0.2879  1.8958
[torch.FloatTensor of size 5x6]

Variable containing:
-0.0460 -3.3375 -4.6613
-4.0205 -0.0306 -4.4052
-4.7923 -4.1103 -0.0250
-0.1025 -2.5474 -3.9579
-3.3321 -0.0433 -5.

...

Variable containing:
-4.3117 -0.0260 -4.3995
-4.7197 -4.2477 -0.0235
-0.0233 -4.0611 -5.1429
-4.1714 -0.0184 -5.8915
[torch.FloatTensor of size 4x3]

Variable containing:
 1.0247  0.0618  0.6400 -1.0456  1.4447  0.1576
-0.5189  2.6226  0.5662  0.8337  2.4017  0.4652
-0.4781  1.2692 -0.4274  0.4128  1.8415  2.2146
 0.1763 -0.4259  1.1740  2.4934 -1.2568 -0.1662
-2.5859 -0.6520  0.8439 -1.1402 -0.1599  1.9482
[torch.FloatTensor of size 5x6]

Variable containing:
-0.0298 -3.7518 -5.1318
-4.4986 -0.0200 -4.7471
-5.1988 -4.5309 -0.0164
-0.0473 -3.2930 -4.6999
-3.8205 -0.0253 -5.7832
[torch.FloatTensor of size 5x3]

Variable containing:
-1.4926  1.2974  0.7086  0.5723  0.2552  0.4016
 0.6243  1.0043 -1.3354  0.7511 -0.0903  0.6954
 0.1775 -0.1601 -0.3994 -0.9537  0.6888 -0.8482
-0.6372  1.2458  0.8102  0.3547 -0.9508  0.6402
[torch.FloatTensor of size 4x6]

Variable containing:
-4.3194 -0.0258 -4.4058
-4.7265 -4.2534 -0.0233
-0.0231 -4.0698 -5.1520
-4.1791 -0.0182 -5.8989
[torch.FloatTensor 

Variable containing:
-0.0250 -3.9258 -5.3055
-4.6998 -0.0168 -4.8829
-5.3766 -4.7060 -0.0138
-0.0340 -3.6193 -5.0175
-4.1228 -0.0187 -6.0724
[torch.FloatTensor of size 5x3]

Variable containing:
-1.5055  1.3094  0.7196  0.5637  0.2368  0.4181
 0.6110  1.0190 -1.3424  0.7756 -0.0762  0.7083
 0.1525 -0.1424 -0.3781 -0.9521  0.7015 -0.8256
-0.6560  1.2591  0.8657  0.3961 -0.9302  0.6558
[torch.FloatTensor of size 4x6]

Variable containing:
-4.5149 -0.0216 -4.5668
-4.9009 -4.3992 -0.0199
-0.0184 -4.2947 -5.3868
-4.3821 -0.0149 -6.0861
[torch.FloatTensor of size 4x3]

Variable containing:
 1.0168  0.0745  0.6661 -1.0586  1.4622  0.1825
-0.5336  2.6347  0.5787  0.8469  2.4195  0.4815
-0.4687  1.2651 -0.4369  0.4090  1.8493  2.2326
 0.1679 -0.4173  1.1679  2.4656 -1.2105 -0.1527
-2.6121 -0.6273  0.8686 -1.0949 -0.1130  1.9610
[torch.FloatTensor of size 5x6]

Variable containing:
-0.0248 -3.9319 -5.3114
-4.7068 -0.0167 -4.8877
-5.3829 -4.7121 -0.0137
-0.0336 -3.6308 -5.0287
-4.1338 -0.0185 -6.

Variable containing:
 1.0143  0.0847  0.6886 -1.0698  1.4776  0.2048
-0.5458  2.6448  0.5888  0.8576  2.4341  0.4951
-0.4613  1.2629 -0.4446  0.4022  1.8547  2.2476
 0.1603 -0.4076  1.1643  2.4442 -1.1716 -0.1403
-2.6344 -0.6095  0.8888 -1.0615 -0.0780  1.9732
[torch.FloatTensor of size 5x6]

Variable containing:
-0.0212 -4.0913 -5.4579
-4.8846 -0.0143 -5.0092
-5.5478 -4.8688 -0.0116
-0.0250 -3.9257 -5.3147
-4.4153 -0.0140 -6.3202
[torch.FloatTensor of size 5x3]

Variable containing:
-1.5171  1.3201  0.7297  0.5576  0.2176  0.4332
 0.5988  1.0327 -1.3484  0.7961 -0.0629  0.7204
 0.1292 -0.1265 -0.3539 -0.9481  0.7130 -0.8049
-0.6737  1.2709  0.9181  0.4347 -0.9116  0.6704
[torch.FloatTensor of size 4x6]

Variable containing:
-4.6953 -0.0182 -4.7168
-5.0664 -4.5373 -0.0172
-0.0148 -4.5080 -5.6038
-4.5778 -0.0123 -6.2585
[torch.FloatTensor of size 4x3]

Variable containing:
 1.0142  0.0850  0.6893 -1.0702  1.4781  0.2056
-0.5462  2.6451  0.5892  0.8580  2.4346  0.4955
-0.4611  1.2629 -0.


Variable containing:
-0.0183 -4.2442 -5.5881
-5.0493 -0.0124 -5.1242
-5.7069 -5.0160 -0.0100
-0.0191 -4.1941 -5.5726
-4.6679 -0.0109 -6.5287
[torch.FloatTensor of size 5x3]

Variable containing:
-1.5271  1.3295  0.7389  0.5538  0.1995  0.4467
 0.5881  1.0450 -1.3534  0.8118 -0.0507  0.7314
 0.1085 -0.1128 -0.3298 -0.9432  0.7232 -0.7864
-0.6896  1.2811  0.9645  0.4676 -0.8955  0.6839
[torch.FloatTensor of size 4x6]

Variable containing:
-4.8574 -0.0157 -4.8533
-5.2189 -4.6643 -0.0150
-0.0122 -4.6988 -5.7948
-4.7578 -0.0103 -6.4150
[torch.FloatTensor of size 4x3]

Variable containing:
 1.0141  0.0940  0.7095 -1.0806  1.4926  0.2272
-0.5572  2.6543  0.5982  0.8675  2.4477  0.5059
-0.4547  1.2618 -0.4517  0.3934  1.8589  2.2617
 0.1532 -0.3974  1.1628  2.4261 -1.1365 -0.1283
-2.6545 -0.5946  0.9063 -1.0350 -0.0493  1.9878
[torch.FloatTensor of size 5x6]

Variable containing:
-0.0182 -4.2495 -5.5924
-5.0549 -0.0124 -5.1281
-5.7123 -5.0210 -0.0100
-0.0189 -4.2029 -5.5811
-4.6762 -0.0108 -6

Variable containing:
-0.0161 -4.3770 -5.6926
-5.1878 -0.0110 -5.2229
-5.8445 -5.1414 -0.0088
-0.0154 -4.4101 -5.7765
-4.8693 -0.0090 -6.6971
[torch.FloatTensor of size 5x3]

Variable containing:
-1.5354  1.3373  0.7466  0.5517  0.1836  0.4581
 0.5791  1.0555 -1.3574  0.8228 -0.0403  0.7407
 0.0915 -0.1015 -0.3086 -0.9385  0.7317 -0.7709
-0.7030  1.2894  1.0027  0.4933 -0.8824  0.6955
[torch.FloatTensor of size 4x6]

Variable containing:
-4.9946 -0.0138 -4.9702
-5.3506 -4.7740 -0.0133
-0.0104 -4.8570 -5.9513
-4.9120 -0.0088 -6.5497
[torch.FloatTensor of size 4x3]

Variable containing:
 1.0150  0.1013  0.7254 -1.0895  1.5051  0.2460
-0.5663  2.6620  0.6055  0.8754  2.4585  0.5126
-0.4495  1.2614 -0.4573  0.3852  1.8617  2.2727
 0.1478 -0.3891  1.1631  2.4132 -1.1102 -0.1189
-2.6695 -0.5837  0.9190 -1.0172 -0.0291  2.0018
[torch.FloatTensor of size 5x6]

Variable containing:
-0.0160 -4.3820 -5.6963
-5.1929 -0.0110 -5.2265
-5.8496 -5.1460 -0.0087
-0.0153 -4.4179 -5.7837
-4.8765 -0.0089 -6.